In [ ]:
import multiprocessing
import os
import psutil

logical_cpu_count = multiprocessing.cpu_count()
print(f"Total logical CPUs: {logical_cpu_count}")

try:
    allowed_cpu_count = len(os.sched_getaffinity(0))
    print(f"Allowed CPUs for the process: {allowed_cpu_count}")
except:
    allowed_cpu_count = psutil.cpu_count(logical=True)
    print(f"Allowed CPUs for the process: {allowed_cpu_count}")

In [ ]:
import scanpy as sc
sc.settings.n_jobs = -1
# import h5py
# import umap
import numpy as np
import pandas as pd
import seaborn as sns
import anndata as ad
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
import pydeseq2.preprocessing as preprocess
import matplotlib as plt

Preprocessed UMAP data in MTX form comes from the following:
- [human liver samples](https://singlecell.broadinstitute.org/single_cell/study/SCP2154/identification-of-a-broadly-fibrogenic-macrophage-subset-induced-by-type-3-inflammation-human-liver-fibrosis-scrnaseq-atlas#/)
- [mouse liver samples](https://singlecell.broadinstitute.org/single_cell/study/SCP2053/identification-of-a-broadly-fibrogenic-macrophage-subset-induced-by-type-3-inflammation-murine-liver-fibrosis-scrnaseq-atlas?#study-summary )

In [ ]:
# Load the filtered feature/cell matrix from HDF5
adata = sc.read_10x_mtx(
    "../data/SCP2154_human/expression/64130e92cc37d750c0a62f14",
    var_names="gene_symbols",  # uses 2nd column
    make_unique=True,
    cache=True,
)
adata.obs['sample'] = adata.obs.index
adata.obs
# barcodes = pd.read_csv("../data/SCP2154_human/expression/64130e92cc37d750c0a62f14//barcodes_raw.tsv")
# features = pd.read_csv(
#     "../data/SCP2154_human"/expression/64130e92cc37d750c0a62f14//features_raw.tsv"
# )

In [ ]:
metadata = pd.read_csv("../data/SCP2154_human/metadata/2022_SI_human_liver_allcells_metadata_file.txt", sep="\t").set_index("NAME").iloc[1:,[1, 5, 14]]
metadata

In [ ]:
# Suppose adata.obs has a column 'cell_id' that matches df_to_add['cell_id']
adata.obs = adata.obs.merge(metadata, left_on='sample', right_on='NAME', how='left')
adata.obs

In [ ]:
adata.var.index[adata.var.index.str.contains("MIR122HG")]

## Whole sample preprocessing + cell type extraction

In [ ]:
adata.obs

In [ ]:
# mitochondrial genes, MT for human Mt for mice
adata.var["mt"] = adata.var_names.str.startswith("MT-")
# ribosomal genes
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt', 'ribo'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(
adata,
["n_genes_by_counts", "total_counts", "pct_counts_mt", "pct_counts_ribo"],
jitter=0.4,
multi_panel=True,
) 

adata = adata[adata.obs.pct_counts_mt <= 20]
adata = adata[adata.obs.pct_counts_ribo < 1]

sc.pp.filter_cells(adata, min_genes=500) 
sc.pp.filter_genes(adata, min_cells=3) 

sc.pl.violin(
adata,
["n_genes_by_counts", "total_counts", "pct_counts_mt", "pct_counts_ribo"],
jitter=0.4,
multi_panel=True,
) 

In [ ]:
sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt")

In [ ]:
sc.pp.scrublet(adata, batch_key="donor_id") #can be configured to factor in batches

# normalize
adata.layers["counts"] = adata.X.copy()
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)

### start from here after preprocessing

In [ ]:
# adata.write("cache/processed_and_normalized_mt.h5ad")
adata = sc.read("cache/processed_and_normalized_mt.h5ad")

In [ ]:
# Scale the data and perform PCA
sc.pp.scale(adata, max_value=50)
sc.tl.pca(adata, svd_solver='arpack')

In [ ]:
# ---------------------------- Human Data Settings --------------------------- #
key = "n53_pc10"
resolution = 0.2
leidan_key = f"leiden_res{resolution}_{key}"
sc.pp.neighbors(adata, n_neighbors=53, n_pcs=10, key_added=key)

# Plot UMAP
sc.tl.leiden(adata, flavor="igraph", resolution=resolution, key_added=leidan_key, neighbors_key=key)
sc.tl.umap(adata, neighbors_key=key)
sc.pl.umap(adata, legend_loc="on data", color=[leidan_key])

In [ ]:
sc.pl.umap(adata, color=["MIR122HG", "ALB", "APOA2", "CYP2E1"], frameon=False, vmax = 10, vmin=0, ncols=2, neighbors_key=key,save="_markers_lnc122_projections_mt2.png")

In [ ]:
sc.pl.dotplot(adata, ['ALB', 'APOA2', 'CYP2E1', 'ASS1', 'PCK1', 'TTR', 'MIR122HG'], groupby=leidan_key, save="_markers_lnc122_mt2.pdf")

In [ ]:
# sc.tl.rank_genes_groups(adata, groupby=leidan_key, method="t-test")
# sc.get.rank_genes_groups_df(adata, group="15").head(20)["names"]

In [ ]:
# hepatocytes = adata[(adata.obs[leidan_key] == '10') | (adata.obs[leidan_key] == '11')].copy()
# hepatocytes.X = hepatocytes.layers["counts"] #make sure to use raw data

hepatocytes = adata[(adata.obs[leidan_key] == '9') | (adata.obs[leidan_key] == '13')].copy()

# # hvg cleaned data
# metadata = adata.obs
# adata = sc.read("cache/processed_and_normalized_mt.h5ad")
# adata.X = adata.layers["counts"].copy()
# adata.obs = adata.obs = metadata.reindex(adata.obs_names)

In [ ]:
hepatocytes.shape

In [ ]:
print(hepatocytes.obs[leidan_key].dtype)

In [ ]:
# # # Split leidan clusters

# hepatocytes.obs['donor_id'] = hepatocytes.obs['donor_id'].astype(str)

# mask = (hepatocytes.obs['donor_id'] == "P8") & (hepatocytes.obs[leidan_key] == '10')
# hepatocytes.obs.loc[mask, 'donor_id'] = "P8_10"

# mask = (hepatocytes.obs['donor_id'] == "P8") & (hepatocytes.obs[leidan_key] == '12')
# hepatocytes.obs.loc[mask, 'donor_id'] = "P8_12"

In [320]:
# get the dominant leiden cluster for each sample
pd.set_option('display.max_rows', None)
hepatocytes.obs.groupby("donor_id")[leidan_key].agg(lambda x: x.value_counts().idxmax())

donor_id
C41              9
C58_RESEQ       13
C70_RESEQ        9
C72_RESEQ       13
H02              9
H13              9
H16              9
H18              9
H22              9
H23             13
H25              9
P2               9
P3               9
P4               9
P5               9
P6               9
P7               9
P8              13
P9              13
P13              9
P14              9
cirr1            9
cirr2            9
cirr3            9
macparland.1    13
macparland.2     9
macparland.3     9
macparland.4    13
macparland.5     9
norm1            9
norm3            9
norm4            9
norm5            9
zhao9            9
Name: leiden_res0.2_n53_pc10, dtype: category
Categories (2, object): ['9', '13']

In [ ]:
# # aggregate small clusters <50. if still <50 drop

# # Merge sampleB and sampleC into 'sampleBC'
# hepatocytes.obs['pseudobulk_id'] = hepatocytes.obs['donor_id'].astype(str)

# hepatocytes.obs.loc[hepatocytes.obs['donor_id'].isin(['norm1', 'norm3', 'norm4', 'norm5']), 'pseudobulk_id'] = 'norm'
# hepatocytes.obs.loc[hepatocytes.obs['donor_id'].isin(['macparland.1', 'macparland.5', "macparland.4"]), 'pseudobulk_id'] = 'macparland.1.4.5'
# hepatocytes.obs.loc[hepatocytes.obs['donor_id'].isin(['H02', 'H13', 'H16', 'H18', 'H22', "H25"]), 'pseudobulk_id'] = 'H_study'
# hepatocytes.obs.loc[hepatocytes.obs['donor_id'].isin(['P13', 'P2', 'P3', 'P4', 'P5', 'P6']), 'pseudobulk_id'] = 'P_study'
# hepatocytes.obs.loc[hepatocytes.obs['donor_id'].isin(['cirr1', 'cirr2', 'cirr3']), 'pseudobulk_id'] = 'cirr_study'

# # zhao drop bc only has two cells
# hepatocytes = hepatocytes[hepatocytes.obs['pseudobulk_id'] != 'zhao9'].copy()
# hepatocytes = hepatocytes[hepatocytes.obs['pseudobulk_id'] != 'norm'].copy()

In [ ]:
gene = "MIR122HG"

# Extract expression values and convert to dense (flattened)
expr = hepatocytes[:, gene].X.toarray().flatten()

# Define percentiles
low_thresh = np.percentile(expr, 25)
high_thresh = np.percentile(expr, 75)
# low_thresh = 0

# Assign labels based on thresholds
def assign_level(x):
    if x <= low_thresh:
        return "Low"
    elif x < high_thresh:
        return "Mid"
    else:
        return "High"

hepatocytes.obs["MIR122HG_level"] = [assign_level(x) for x in expr]


In [ ]:
# import random
pbs = []
key_12 = ["C58_RESEQ", "C72_RESEQ", "H23", "P8", "P9"]
# key_13 = ["C58_RESEQ", "C72_RESEQ", "H11", "H23", "P8", "P9", "macparland.1", "macparland.4"]
for donor_id in hepatocytes.obs["donor_id"].unique():
    samp_hepatocytes = hepatocytes[hepatocytes.obs["donor_id"] == donor_id]
    samp_hepatocytes.X = samp_hepatocytes.layers['counts']

    X_sum = samp_hepatocytes.X.sum(axis = 0).reshape(1, -1)

    rep_adata = sc.AnnData(X = X_sum,
                            var = samp_hepatocytes.var[[]])
    rep_adata.obs_names = [donor_id]
    rep_adata.obs["donor_id"] = samp_hepatocytes.obs["donor_id"].iloc[0]
    rep_adata.obs[leidan_key] = "High" if (donor_id in key_12) else "Low"

    rep_adata.obs
    pbs.append(rep_adata)

    # indices = list(samp_hepatocytes.obs_names)
    # random.shuffle(indices)
    # num_reps = 5 if (donor_id == "15") else 10
    # indices = np.array_split(np.array(indices), num_reps)
    
    # for i, pseudo_rep in enumerate(indices):
    
    #     rep_adata = sc.AnnData(X_sum, var = samp_hepatocytes[indices[i]].var[[]])

    #     rep_adata.obs_names = [donor_id + '_' + str(i)]
    #     rep_adata.obs[donor_id] = samp_hepatocytes.obs[donor_id].iloc[0]
    #     rep_adata.obs['replicate'] = i

    #     pbs.append(rep_adata)

In [ ]:
hepatocytes = sc.concat(pbs)

In [ ]:
hepatocytes.obs

In [ ]:
# for gct only
hepatocytes.obs["sample_name"] = hepatocytes.obs_names + "_" + hepatocytes.obs["leiden_res0.2_n50_pc10"]
counts = pd.DataFrame(hepatocytes.X, columns = hepatocytes.var_names, index=hepatocytes.obs["sample_name"])
norm_counts, size_factors = preprocess.deseq2_norm(counts)

norm_counts.T.to_csv("/home/hjang620/20250603_Anh_HCC/scrna/notebook_scripts/deseq2_norm.gct", sep="\t")

In [ ]:
counts = pd.DataFrame(hepatocytes.X, columns = hepatocytes.var_names)
counts.T.to_csv("intermediate/counts_intermediate.csv")
hepatocytes.obs.to_csv("intermediate/metadata_intermediate.csv")
print(leidan_key)

In [ ]:
counts = pd.DataFrame(hepatocytes.X, columns = hepatocytes.var_names)
dds = DeseqDataSet(
    counts = counts,
    metadata=hepatocytes.obs,
    design_factors=leidan_key)
dds

In [ ]:
sc.pp.filter_genes(dds, min_cells = 1)
dds

In [ ]:
dds.deseq2()

In [ ]:
sc.tl.pca(dds)
sc.pl.pca(dds, color = leidan_key, size = 200)

In [ ]:
stat_res = DeseqStats(dds, n_cpus=8, contrast=(leidan_key, "Low", 'High'))
stat_res.summary()

In [ ]:
de  = stat_res.results_df
# de.sort_values('stat', ascending = False)
de.to_csv("leidan_ranking_mt.csv")

## GSEA analysis!

In [ ]:
hepatocytes.obs["MIR122HG_level"].value_counts()

In [ ]:
import os
gene_sets = {}
for file in ["/home/hjang620/20250603_Anh_HCC/reference_files/human/Homo_sapiens_HALLMARK_OXIDATIVE_PHOSPHORYLATION.csv", 
             "/home/hjang620/20250603_Anh_HCC/reference_files/human/Homo_sapiens_KEGG_CITRATE_CYCLE_TCA_CYCLE.csv", 
             "/home/hjang620/20250603_Anh_HCC/reference_files/human/Homo_sapiens_REACTOME_MITOCHONDRIAL_BIOGENESIS.csv"]:
    df = pd.read_csv(file, header=0, index_col=0)
    set = os.path.basename(file).split(".")[0].replace("Homo_sapiens_", "")
    gene_sets[set] = df["gene_symbol"].tolist()

In [ ]:
hepatocytes.obs[leidan_key] = pd.Categorical(hepatocytes.obs[leidan_key], categories=["9", "12"], ordered=True)
indices = hepatocytes.obs.sort_values([leidan_key]).index
hepatocytes = hepatocytes[indices,:]

In [ ]:
hepatocytes.obs['MIR122HG_level']

In [ ]:
# sc resolution gseapy
import gseapy as gp
res = gp.gsea(data=hepatocytes.to_df().T, # row -> genes, column-> samples
        gene_sets=gene_sets,
        cls=hepatocytes.obs["MIR122HG_level"],
        permutation_num=1000,
        permutation_type='phenotype', 
        outdir="sc_res_expr",
        method='signal_to_noise', 
        threads= 16)

In [ ]:
term = res.res2d.Term
axs = res.plot(terms=term[:5])

In [ ]:
sc.tl.rank_genes_groups(hepatocytes, 
                        groupby='MIR122HG_level', 
                        use_raw=False,
                        method='wilcoxon', 
                        groups=["Low"], 
                        reference='High')

In [ ]:
result = hepatocytes.uns['rank_genes_groups']
groups = result['names'].dtype.names
degs = pd.DataFrame(
    {group + '_' + key: result[key][group]
    for group in groups for key in ['names','scores', 'pvals','pvals_adj','logfoldchanges']})

In [ ]:
degs.head()

In [ ]:
import gseapy as gp
pre_res = gp.prerank(degs.loc[:,['Low_names', 'Low_logfoldchanges']], gene_sets=gene_sets)